In [2]:
import os
import json

import pandas as pd
from dotenv import load_dotenv


path = "../data/raw/market_products.csv"

# Lectura del archivo csv

In [3]:
dtypes = {
    "sku": str,
    "gtin": str
}
df = pd.read_csv(path, dtype=dtypes)
df.sample()

,sku,gtin,specifications,brand,description,product_name,review_tags,category_name,root_category_name,main_image,unit,sizes,colors,other_attributes,categories
242,3051626681,463051626689,"[{""name"":""Features"",""value"":""Easy Care""},{""nam...",Woman Within,Woman Within Women's Plus Size Perfect Printed...,Woman Within Women's Plus Size Perfect Printed...,"[""Fit (106)"",""Comfort (71)"",""Color (64)"",""Leng...",Plus Size Tops,Clothing,"""https://i5.walmartimages.com/seo/Woman-Within...",NaN,"[""1X (22-24)"",""2X (26-28)"",""3X (30-32)"",""4X (3...","[""Azure Paisley"",""Black Bandana Paisley"",""Blac...","[{""name"":""Fabric Care Instructions"",""value"":""M...","[""Clothing"",""Womens Plus"",""Plus Size Tops"",""Pl..."


# Hallazgos iniciales

* La mayor cantidad de mercancias se relacionan con la ropa
* SKU es un valor unico que se puede usar en el futuro para referirse a mercancias especificas
* Hay alguna informacion de unidades pero esta muy incompleta, las unidades son muy inconsistentes
* Las especificaciones tienen gran cantidad de informacion en strings, que representan listas de diccionarios. Incluyen peso y dimensiones a veces.

# Asignacion de volumen

Ideas para el primer acercamiento con complejidad incremental:
1. Extraer la informacion de peso y volumen presente
2. Uniformar las cantidades
3. Completar con valores por defecto (marcar estos productos)
3. Desarrollar modelos que predigan peso y volumen (k-nearest neighbors, embeddings y SMV o ANN, uso de APIs de LLMs)

In [9]:
def safe_parse(x):
    if pd.notna(x):
        return json.loads(x)

def summarize_names(series):
    summary = []
    for specs in series.dropna():
        if isinstance(specs, list):
            for s in specs:
                if isinstance(s, dict) and "name" in s:
                    summary.append(s["name"])
    return summary

df['specifications_parsed'] = df['specifications'].apply(safe_parse)
df['other_attributes_parsed'] = df['other_attributes'].apply(safe_parse)

In [4]:
def get_weight_or_vol(values: dict, search_key) -> str | None:
    """Busca la etiqueta correcta con peso o volumen en la lista de dictionarios"""
    try:
        return next(d['value'] for d in values if d.get('name') == search_key)
    except StopIteration:
        return None

In [11]:
# explorando las especificaciones que tienen informacion sobre volumen
col_name = "specifications_parsed"
search_key = "Assembled Product Dimensions (L x W x H)"
has_vol = df[col_name].astype(str).str.contains(search_key, na=False, case=False, regex=False)

In [12]:
# crear copia de df y usar sku como indice
vols = df.copy()
vols = vols.set_index(keys=["sku"])
has_vol = vols[col_name].astype(str).str.contains(search_key, na=False, case=False, regex=False)

In [13]:
col_l = "size_l"
col_w = "size_w"
col_h = "size_h"
col_units = "size_units"
# extrae informacion de volumen de la columna de especificaciones
vols_sr = vols[col_name].apply(
    lambda x: get_weight_or_vol(x, search_key=search_key)
)
vols_sr.value_counts().sample(10)

specifications_parsed
2.80 x 2.80 x 2.70 Inches       1
60.00 x 24.00 x 0.10 Inches     1
80.00 x 90.00 x 1.00 Inches     1
92.00 x 66.00 x 1.00 Inches     2
92.00 x 68.00 x 2.00 Inches     2
3.50 x 1.00 x 5.60 Inches       1
83.10 x 60.60 x 44.90 Inches    1
14.75 x 13.31 x 11.44 Inches    1
36.00 x 24.00 x 0.50 Inches     1
2.25 x 8.75 x 2.25 Inches       1
Name: count, dtype: int64

In [14]:
vols = vols_sr.str.extractall(
    fr"(?P<{col_l}>[\d\.]+) x (?P<{col_w}>[\d\.]+) x (?P<{col_h}>[\d\.]+) (?P<{col_units}>\w+)"
)
vols = vols.droplevel(level=1)  # eliminar indice de match
vols.sample(3)

,size_l,size_w,size_h,size_units
sku,,,,
145946079,9.80,2.90,1.90,Inches
827790002,2.36,5.12,9.10,Inches
5168273880,13.00,8.30,2.50,Inches


In [15]:
# usar valores numericos
vols[col_h] = pd.to_numeric(vols[col_h])
vols[col_w] = pd.to_numeric(vols[col_w])
vols[col_l] = pd.to_numeric(vols[col_l])

## Uniformar volumen

In [16]:
vols[col_units].value_counts(dropna=False)

size_units
Inches    282
Feet        9
Name: count, dtype: int64

In [17]:
vols.query("size_units == 'Feet'")

,size_l,size_w,size_h,size_units
sku,,,,
5354771215,1.00,1.00,1.00,Feet
1796059258,0.98,0.66,0.07,Feet
1979723992,0.16,0.13,0.03,Feet
1980636541,0.16,0.16,0.03,Feet
1577779507,10.00,10.00,10.00,Feet
968072447,0.43,0.23,0.07,Feet
939136581,0.59,0.33,0.10,Feet
786691113,1.00,1.00,1.00,Feet
178944020,0.66,0.49,0.10,Feet


In [18]:
# convertir de pies a pulgadas
vols.loc[vols[col_units] == "Feet", [col_h, col_w, col_l]] *= 12

In [19]:
vols.query("size_units == 'Feet'")

,size_l,size_w,size_h,size_units
sku,,,,
5354771215,12.00,12.00,12.00,Feet
1796059258,11.76,7.92,0.84,Feet
1979723992,1.92,1.56,0.36,Feet
1980636541,1.92,1.92,0.36,Feet
1577779507,120.00,120.00,120.00,Feet
968072447,5.16,2.76,0.84,Feet
939136581,7.08,3.96,1.20,Feet
786691113,12.00,12.00,12.00,Feet
178944020,7.92,5.88,1.20,Feet


In [20]:
vols[col_units] = "in"
df = df.merge(vols.reset_index(), how="left", on="sku")
df.sample()

,sku,gtin,specifications,brand,description,product_name,review_tags,category_name,root_category_name,main_image,...,sizes,colors,other_attributes,categories,specifications_parsed,other_attributes_parsed,size_l,size_w,size_h,size_units
333,799517514,880021737600,"[{""name"":""Features"",""value"":""Hypoallergenic, W...",SGI bedding,Enhances Bedroom Decor:&nbsp; Our bedskirt ...,SGI Bedding 14 Inch Wrap Around Bed Skirt | Mi...,"[""Color (6)""]",Bed Skirts,Home,"""https://i5.walmartimages.com/seo/SGI-Bedding-...",...,"[""California King- 14\"""",""Full- 14\"""",""King- 1...","[""Black"",""Blood Red"",""Burgundy"",""Chocolate"",""D...","[{""name"":""Fabric Care Instructions"",""value"":""M...","[""Home"",""Bedding"",""Bed Skirts""]","[{'name': 'Features', 'value': 'Hypoallergenic...","[{'name': 'Fabric Care Instructions', 'value':...",78.0,80.0,14.0,in


In [21]:
mean_vols = df.groupby(["root_category_name"]).agg(
    {
        "size_l": "mean",
        "size_w": "mean",
        "size_h": "mean",
    }
)
mean_vols

,size_l,size_w,size_h
root_category_name,,,
Arts Crafts & Sewing,NaN,NaN,NaN
Auto & Tires,NaN,NaN,NaN
Baby,8.685000,8.300000,7.927500
Beauty,5.503333,6.751429,12.579524
Clothing,10.639574,9.226170,6.263191
Collectibles,7.000000,5.900000,1.700000
Electronics,0.980000,2.240000,4.800000
Food,4.147273,4.129091,5.817273
Health and Medicine,4.750000,6.105000,5.215000


In [ ]:
# drop rows with na, and round
pathout_json = "../data/processed/mean_volumes.json"
mean_vols.dropna().round(1).to_json(pathout_json, orient="index", indent=4)

# pathout_json = "../data/processed/guessed_volumes.json"
# mean_vols.round(1).to_json(pathout_json, orient="index", indent=4)

# Asignacion de peso

In [23]:
# explorando las especificaciones que tienen informacion sobre el peso
col_name = "specifications_parsed"
search_key = "Assembled Product Weight"
col_weight = "weight_value"
col_weight_unit = "weight_unit"
has_weight = df[col_name].astype(str).str.contains(search_key, na=False, case=False)
df[has_weight][col_name].sample().values

array([list([{'name': 'Features', 'value': 'Lightweight'}, {'name': 'Country of Origin - Textiles', 'value': 'Imported'}, {'name': 'Clothing Size', 'value': '2XL'}, {'name': 'Brand', 'value': 'Betiyuaoe'}, {'name': 'Gender', 'value': 'Female'}, {'name': 'Assembled Product Weight', 'value': '0.22 lb'}, {'name': 'Manufacturer', 'value': 'Follure'}, {'name': 'Color', 'value': 'Beige'}, {'name': 'Assembled Product Dimensions (L x W x H)', 'value': '8.27 x 0.39 x 5.51 Inches'}])],
      dtype=object)

In [24]:
# crear copia de df y usar sku como indice
weights = df.copy()
weights = weights.set_index(keys=["sku"])
has_weight = weights[col_name].astype(str).str.contains(search_key, na=False, case=False)

In [25]:
# extraer informacion de peso
weights_sr = weights[col_name].apply(
    lambda x: get_weight_or_vol(x, search_key=search_key)
)
weights = weights_sr.str.extractall(
    rf"(?P<{col_weight}>[\d\.]+) ?(?P<{col_weight_unit}>\w+)"
)
weights[col_weight] = pd.to_numeric(weights[col_weight])
weights[col_weight_unit] = weights[col_weight_unit].str.lower()
weights.sample(3)

,,weight_value,weight_unit
sku,match,,
2348411906,0,8.80,lb
37684787,0,0.25,pounds
2599647180,0,2.43,lb


In [26]:
weights[col_weight_unit].value_counts()

weight_unit
lb        160
oz         23
lbs        20
pounds      5
ounces      2
kg          1
Name: count, dtype: int64

In [27]:
# uniformar etiquetas
normalize_units = {
    "lbs": "lb",
    "pounds": "lb",
    "ounces": "oz",
}
weights[col_weight_unit] = weights[col_weight_unit].replace(normalize_units)
weights = weights.droplevel(level=1)  # remueve el indice extra de extractall
weights.sample(3)

,weight_value,weight_unit
sku,,
2200709686,6.00,lb
45696491,0.52,lb
5158180497,0.10,lb


In [28]:
df = df.merge(weights.reset_index(), how="left", on="sku")
df.sample()

,sku,gtin,specifications,brand,description,product_name,review_tags,category_name,root_category_name,main_image,...,other_attributes,categories,specifications_parsed,other_attributes_parsed,size_l,size_w,size_h,size_units,weight_value,weight_unit
516,939136581,779003679390,"[{""name"":""Country of Origin - Textiles"",""value...",LETDIOSTO,LETDIOSTO Women's Plus Size Tops Short Sleeve ...,LETDIOSTO Women's Plus Size Tops Short Sleeve ...,"[""Fit (21)"",""Material (15)"",""Color (15)"",""Size...",Plus Size Tops,Clothing,"""https://i5.walmartimages.com/seo/LETDIOSTO-Wo...",...,"[{""name"":""Fabric Care Instructions"",""value"":""R...","[""Clothing"",""Womens Plus"",""Plus Size Tops"",""Pl...","[{'name': 'Country of Origin - Textiles', 'val...","[{'name': 'Fabric Care Instructions', 'value':...",7.08,3.96,1.2,in,6.26,oz


## Completar peso

In [29]:
df.query("weight_value.notna()").root_category_name.value_counts()

root_category_name
Home                    133
Clothing                 29
Patio & Garden            9
Personal Care             8
Pets                      8
Health and Medicine       5
Food                      5
Home Improvement          3
Baby                      2
Beauty                    2
Household Essentials      2
Toys                      2
Collectibles              1
Arts Crafts & Sewing      1
Sports & Outdoors         1
Name: count, dtype: int64

In [162]:
df.query("weight_value.isna()").specifications.values

array(['[{"name":"Features","value":"Easy Care"},{"name":"Fabric Content","value":"95% Polyester"},{"name":"Gender","value":"Female"},{"name":"Brand","value":"Catherines"}]',
       '[{"name":"Country of Origin - Textiles","value":"Imported"},{"name":"Brand","value":"SANMADROLA"},{"name":"Material","value":"70% Cotton and 30% Polyester Fibers"},{"name":"Manufacturer Part Number","value":"WM2-GTSFT01L2-HUI-L"},{"name":"Color","value":"Gray"},{"name":"Manufacturer","value":"SANMADROLA"}]',
       '[{"name":"Features","value":"Lightweight, Stretch Fabric, Shaping"},{"name":"Country of Origin - Textiles","value":"Imported"},{"name":"Clothing Size","value":"4XL"},{"name":"Fabric Content","value":"Nylon"},{"name":"Brand","value":"HAWEE"},{"name":"Gender","value":"Female"},{"name":"Color","value":"Black"},{"name":"Manufacturer","value":"HAWEE"}]',
       '[{"name":"Country of Origin - Textiles","value":"Imported"},{"name":"Clothing Size","value":"5"},{"name":"Fabric Content","value":"100% 60%

In [30]:
mean_weigts = df.groupby(["root_category_name"]).agg({"weight_value": "mean"})
mean_weigts

,weight_value
root_category_name,
Arts Crafts & Sewing,2.880000
Auto & Tires,NaN
Baby,2.205000
Beauty,0.359000
Clothing,4.438276
Collectibles,1.100000
Electronics,NaN
Food,12.851980
Health and Medicine,7.000000


In [31]:
# drop rows with na, and round
pathout_json = "../data/processed/mean_weights.json"
mean_weigts.dropna().round(1).to_json(pathout_json, orient="index", indent=4)
# pathout_json = "../data/processed/guessed_weights.json"
# mean_weigts.round(1).to_json(pathout_json, orient="index", indent=4)

Faltantes:

* Normalizar valores y unidades de peso
* Desarrollar modelos avanzados

# test api

In [168]:
from google import genai

load_dotenv()
api_key = os.getenv("api_key")
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""What is the average weight of a piece of clothing.""",
)
print(response.text)

That's a great question, but there isn't a single, universally "average" weight for a piece of clothing because the weight varies enormously based on several factors:

1.  **Type of Garment:** A sock is vastly different from a winter coat.
2.  **Material:** Cotton, wool, denim, silk, polyester, linen all have different densities and weights.
3.  **Size:** A small T-shirt weighs less than an XL T-shirt.
4.  **Thickness/Construction:** A light summer dress vs. a heavy, lined winter dress.

However, I can give you some typical ranges for common items:

*   **Very Light (20-100 grams / 0.7-3.5 oz):**
    *   Underwear, socks, silk scarf, thin camisole.
*   **Light (100-300 grams / 3.5-10.5 oz):**
    *   T-shirt, tank top, light blouse, thin pajamas.
*   **Medium (300-800 grams / 10.5 oz - 1.7 lbs):**
    *   Jeans, dress shirt, light sweater, summer dress, tracksuit bottoms, light jacket.
*   **Heavy (800 grams - 2 kg / 1.7 - 4.4 lbs):**
    *   Denim jacket, thick sweater, winter jeans, 